# Question 1: RNN

In [1]:
import itertools
import math
import time

import torch

from models.RNNModel import RNNModel
from training_testing.training import train_model
from utils.batchifier import batchify
from utils.tokenizer import Vocabulary, process_corpus, read_corpus

## Read and Process Data

In [2]:
batch_size = 32

In [3]:
train_file_path = "data/wiki2.train.txt"
valid_file_path = "data/wiki2.valid.txt"

In [4]:
train_processed_tokens = process_corpus(train_file_path)
train_vocab = Vocabulary()
train_vocab.build_vocab(train_processed_tokens)
train_numericalized_tokens = train_vocab.numericalize(train_processed_tokens)
train_numericalized_tokens.size()

torch.Size([2088629])

In [5]:
train_batched_data = batchify(train_numericalized_tokens, batch_size)
train_batched_data.shape

torch.Size([65269, 32])

In [6]:
valid_processed_tokens = read_corpus(valid_file_path)
valid_numericalized_tokens = train_vocab.numericalize(valid_processed_tokens)
valid_numericalized_tokens.size()

torch.Size([217647])

In [7]:
valid_batched_data = batchify(valid_numericalized_tokens, batch_size)
valid_batched_data.shape

torch.Size([6801, 32])

## Training

In [8]:
bptt_values = [30, 20, 10]
clip_threshold_values = [0.25]
log_interval = 1000
lr_values = [0.01]
epochs = 22
dropout_values = [0.3, 0.5]
ntokens = train_vocab.size

criterion = torch.nn.CrossEntropyLoss()

In [9]:
hyperparam_combinations = list(
    itertools.product(bptt_values, clip_threshold_values, lr_values, dropout_values)
)

In [ ]:
results = []

for combination in hyperparam_combinations:
    bptt, clip_threshold, lr, dropout = combination

    model = RNNModel(
        ntokens=ntokens, ninp=100, nhid=100, dropout=dropout
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_loss, val_loss, train_ppl, val_ppl = train_model(
        model=model,
        train_batched_data=train_batched_data,
        valid_batched_data=valid_batched_data,
        epochs=epochs,
        batch_size=batch_size,
        bptt=bptt,
        criterion=criterion,
        optimizer=optimizer,
        clip_threshold=clip_threshold,
        log_interval=log_interval,
        lr=lr,
        dropout=dropout,
    )

    results.append(
        {
            "bptt": bptt,
            "clip_threshold": clip_threshold,
            "lr": lr,
            "dropout": dropout,
            "epochs": epochs,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_ppl": train_ppl,
            "val_ppl": val_ppl,
        }
    )

epoch   1:  1000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 42.41 | train loss  5.64 | train ppl   282.18 | valid loss  5.34 | valid ppl   209.40
epoch   1:  2000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 41.30 | train loss  5.06 | train ppl   156.93 | valid loss  5.16 | valid ppl   174.86
epoch   2:  1000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 40.13 | train loss  4.90 | train ppl   134.46 | valid loss  5.12 | valid ppl   166.99
epoch   2:  2000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 39.87 | train loss  4.87 | train ppl   130.57 | valid loss  5.10 | valid ppl   163.58
epoch   3:  1000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 42.32 | train loss  4.82 | train ppl   123.39 | valid loss  5.14 | valid ppl   171.43
epoch   3:  2000/ 2175 batches | lr 0.01 | dropout 0.30 | ms/batch 43.09 | train loss  4.82 | train ppl   124.34 | valid loss  5.10 | valid ppl   163.87


In [ ]:
df_results = pd.DataFrame(results)

## Testing our best model